In [1]:
#importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

classes = dataset.classes
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [2]:
#defining class for CNN
class FlexibleCNN(nn.Module):
    def __init__(self, num_classes, in_channels=3, conv_filters=[16, 32, 64, 128, 256], 
                 kernel_sizes=[3, 3, 3, 3, 3], pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=512, activation='ReLU', batch_norm = True, dropout = 0.0):
        self.conv_filters = conv_filters
        self.kernel_sizes = kernel_sizes
        self.pool_sizes = pool_sizes
        self.dense_units = dense_units
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout = dropout
        
        super(FlexibleCNN, self).__init__()

        #convolutional layers
        self.conv_layers = nn.ModuleList()
        prev_out_size = 224
        in_channels = in_channels
        for out_channels, kernel_size, pool_size in zip(conv_filters, kernel_sizes, pool_sizes):
            conv_layer = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size),
                self.get_activation(activation),
                nn.MaxPool2d(pool_size)
            )
            
            
            if batch_norm:
                conv_layer.add_module(f"batch_norm_{out_channels}", nn.BatchNorm2d(out_channels))
            if dropout > 0.0:
                conv_layer.add_module(f"dropout_{out_channels}", nn.Dropout2d(dropout))
            self.conv_layers.append(conv_layer)
            in_channels = out_channels
            
        def cal_size(stride, padding, kernel_size, prev_size):
            new_size = (prev_size-kernel_size+2*padding)/stride + 1
            return new_size//2
        
        prev_size = 224
        
        for i in range(5):
            new_size1 = cal_size(stride = 1, padding = 0, kernel_size = self.kernel_sizes[i], prev_size= prev_size)
            prev_size = new_size1
        print(new_size1)

        #fully connected layers
        #self.fc1 = nn.Linear(conv_filters[-1] * 7 * 7, dense_units)
        self.fc1 = nn.Linear(self.conv_filters[4] * int(new_size1) * int(new_size1), self.dense_units)
        self.fc2 = nn.Linear(self.dense_units, 10)#num_classes)

    def forward(self, x):
        # Forward pass through convolutional layers
        for conv_layer in self.conv_layers:
            x = conv_layer(x)

        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1)

        # Forward pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def get_activation(self, activation):
        if activation == 'ReLU':
            return nn.ReLU()
        elif activation == 'GELU':
            return nn.GELU()
        elif activation == 'SiLU':
            return nn.SiLU()
        elif activation == 'Mish':
            return nn.Mish()
        else:
            raise ValueError("Invalid activation function")
            
#model = FlexibleCNN(10)
#print(model)

In [3]:
!pip install wandb

In [4]:
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [5]:
wandb.login(key='cd7a6c2259e8886dc269bbf6f0f9e55089d3beeb')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test15',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3], [3,5,5,7,7], [7,7,5,5,3]]
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'batch_norm':{
            'values': ['true','false']
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'num_dense':{
            'values': [128, 256]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2')

Create sweep with ID: s4e8odr0
Sweep URL: https://wandb.ai/prabhat-kumar/Deep_Learning_Assignment2/sweeps/s4e8odr0


In [7]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-act_"+wandb.config.activation+"-ks"+str(wandb.config.kernel_size)+'-nd'+str(wandb.config.num_dense)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org, 
                 kernel_sizes=wandb.config.kernel_size, pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=wandb.config.num_dense, activation=wandb.config.activation).to(device)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

        # Training loop
        for epoch in range(num_epochs):
            total_train = 0
            correct_train = 0
            running_loss = 0
            # Train the model
            model.train()
            for images, labels in train_loader:
                
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # Calculate the training accuracy
                _, predicted = torch.max(outputs, 1)
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                
            train_accuracy = 100*correct_train/total_train
            print(f'Epoch [{epoch+1}/{num_epochs}], Training Accuracy: {train_accuracy:.2f}%')
            wandb.log({'train_loss': round(running_loss / len(train_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # Validate the model
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in val_loader:
                    
                    images = images.to(device)
                    labels = labels.to(device)
            
                    # Forward pass
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            
        
                val_accuracy = 100 * correct / total
                print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')
                wandb.log({'val_accuracy':val_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training')

wandb.agent(sweep_id, function=main,count=10) # calls main function for count number of times.
wandb.finish()

wandb: Agent Starting Run: 2wh7i6rw with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: false
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 32, 32, 32, 32]
wandb: 	kernel_size: [3, 5, 5, 7, 7]
wandb: 	num_dense: 128
wandb: Currently logged in as: prabhat-kumar. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


1.0
Epoch [1/10], Training Accuracy: 22.98%
Epoch [1/10], Validation Accuracy: 28.85%
Epoch [2/10], Training Accuracy: 28.97%
Epoch [2/10], Validation Accuracy: 32.30%
Epoch [3/10], Training Accuracy: 32.38%
Epoch [3/10], Validation Accuracy: 32.05%
Epoch [4/10], Training Accuracy: 34.57%
Epoch [4/10], Validation Accuracy: 32.70%
Epoch [5/10], Training Accuracy: 36.89%
Epoch [5/10], Validation Accuracy: 31.25%
Epoch [6/10], Training Accuracy: 38.45%
Epoch [6/10], Validation Accuracy: 34.85%
Epoch [7/10], Training Accuracy: 40.94%
Epoch [7/10], Validation Accuracy: 36.30%
Epoch [8/10], Training Accuracy: 42.87%
Epoch [8/10], Validation Accuracy: 35.95%
Epoch [9/10], Training Accuracy: 45.32%
Epoch [9/10], Validation Accuracy: 35.75%
Epoch [10/10], Training Accuracy: 47.39%
Epoch [10/10], Validation Accuracy: 34.80%
Finished Training


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▃▄▄▅▅▆▇▇█
train_loss,█▇▆▅▄▄▃▂▂▁
val_accuracy,▁▄▄▅▃▇██▇▇
epoch,10
train_accuracy,47.39
train_loss,1.51
val_accuracy,34.8


wandb: Agent Starting Run: 9lrvaz6y with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: true
wandb: 	dropout: 0.3
wandb: 	filt_org: [32, 32, 32, 32, 32]
wandb: 	kernel_size: [3, 5, 5, 7, 7]
wandb: 	num_dense: 128
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


1.0
Epoch [1/10], Training Accuracy: 22.34%
Epoch [1/10], Validation Accuracy: 26.25%
Epoch [2/10], Training Accuracy: 28.32%
Epoch [2/10], Validation Accuracy: 30.50%
Epoch [3/10], Training Accuracy: 32.37%
Epoch [3/10], Validation Accuracy: 31.45%
Epoch [4/10], Training Accuracy: 34.58%
Epoch [4/10], Validation Accuracy: 32.75%
Epoch [5/10], Training Accuracy: 36.30%
Epoch [5/10], Validation Accuracy: 33.50%
Epoch [6/10], Training Accuracy: 38.17%
Epoch [6/10], Validation Accuracy: 33.20%
Epoch [7/10], Training Accuracy: 40.72%
Epoch [7/10], Validation Accuracy: 34.60%
Epoch [8/10], Training Accuracy: 42.56%
Epoch [8/10], Validation Accuracy: 37.55%
Epoch [9/10], Training Accuracy: 44.98%
Epoch [9/10], Validation Accuracy: 35.95%
Epoch [10/10], Training Accuracy: 48.63%
Epoch [10/10], Validation Accuracy: 36.35%
Finished Training


...

5.0
Epoch [1/10], Training Accuracy: 26.05%
Epoch [1/10], Validation Accuracy: 32.40%
Epoch [2/10], Training Accuracy: 33.97%
Epoch [2/10], Validation Accuracy: 32.70%
Epoch [3/10], Training Accuracy: 39.97%
Epoch [3/10], Validation Accuracy: 35.65%
Epoch [4/10], Training Accuracy: 45.66%
Epoch [4/10], Validation Accuracy: 33.65%
Epoch [5/10], Training Accuracy: 51.14%
Epoch [5/10], Validation Accuracy: 35.00%
Epoch [6/10], Training Accuracy: 59.38%
Epoch [6/10], Validation Accuracy: 34.05%
Epoch [7/10], Training Accuracy: 68.63%
Epoch [7/10], Validation Accuracy: 34.95%
Epoch [8/10], Training Accuracy: 77.20%
Epoch [8/10], Validation Accuracy: 34.55%
Epoch [9/10], Training Accuracy: 84.77%
Epoch [9/10], Validation Accuracy: 32.90%
Epoch [10/10], Training Accuracy: 89.65%
Epoch [10/10], Validation Accuracy: 33.90%
Finished Training


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▃▃▄▅▆▇▇█
train_loss,█▇▇▆▅▄▃▂▂▁
val_accuracy,▁▂█▄▇▅▆▆▂▄
epoch,10
train_accuracy,89.65
train_loss,0.34
val_accuracy,33.9


wandb: Agent Starting Run: az0v4aoj with config:
wandb: 	activation: ReLU
wandb: 	batch_norm: false
wandb: 	dropout: 0.2
wandb: 	filt_org: [32, 64, 128, 256, 512]
wandb: 	kernel_size: [3, 3, 3, 3, 3]
wandb: 	num_dense: 128
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


5.0
Epoch [1/10], Training Accuracy: 19.80%
Epoch [1/10], Validation Accuracy: 24.05%
Epoch [2/10], Training Accuracy: 30.03%
Epoch [2/10], Validation Accuracy: 29.75%
Epoch [3/10], Training Accuracy: 34.48%
Epoch [3/10], Validation Accuracy: 30.45%
Epoch [4/10], Training Accuracy: 37.28%
Epoch [4/10], Validation Accuracy: 30.05%
Epoch [5/10], Training Accuracy: 40.84%
Epoch [5/10], Validation Accuracy: 33.05%
Epoch [6/10], Training Accuracy: 44.66%
Epoch [6/10], Validation Accuracy: 31.60%
Epoch [7/10], Training Accuracy: 50.54%
Epoch [7/10], Validation Accuracy: 32.95%
Epoch [8/10], Training Accuracy: 56.84%
Epoch [8/10], Validation Accuracy: 33.60%
Epoch [9/10], Training Accuracy: 65.55%
Epoch [9/10], Validation Accuracy: 33.00%
Epoch [10/10], Training Accuracy: 74.63%
Epoch [10/10], Validation Accuracy: 32.55%
Finished Training


epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▂▃▃▄▄▅▆▇█
train_loss,█▅▅▅▄▄▃▃▂▁
val_accuracy,▁▅▆▅█▇███▇
epoch,10
train_accuracy,74.63
train_loss,0.75
val_accuracy,32.55


## Testing using best hyperparameter